# Install required libraries

In [1]:
!pip install librosa==0.6.3 numpy soundfile==0.9.0 sklearn 
!pip install -q keras

# Import the libraries

In [2]:
import soundfile # to read audio file
import numpy as np
import librosa # to extract speech features
import glob
import os
import pickle # to save model after training
from sklearn.model_selection import train_test_split # for splitting training and testing
from sklearn.neural_network import MLPClassifier # multi-layer perceptron model
from sklearn.metrics import accuracy_score # to measure how good we are
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Input, Flatten, Dropout, Activation, Dense
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, LSTM
from keras.models import Model
from keras.layers import Input, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

Using TensorFlow backend.


# Mount drive to fetch data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Function to extract features from the audio dataset

In [0]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        if X.ndim >= 2:
            X = np.mean(X, 1)
        sample_rate = sound_file.samplerate
        result = np.array([])
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
    return result

Filter the required emotions - We use only 5 emotions for training 

In [0]:
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions ( feel free to tune this on your need )
AVAILABLE_EMOTIONS = {
    "neutral",
    "calm",
    "happy",
    "sad",
    "angry",
}

# Function to load data and split the training and test data

In [0]:
def load_data(test_size=0.2):
    X, y = [], []
    for file in glob.glob("/content/drive/My Drive/Dataset/*.wav"):
        # get the base name of the audio file
        basename = os.path.basename(file)
        # get the emotion label
        emotion = int2emotion[basename.split("-")[2]]
        # we allow only AVAILABLE_EMOTIONS we set
        if emotion not in AVAILABLE_EMOTIONS:
            continue
        # extract speech features
        features = extract_feature(file, mfcc=True, chroma=False, mel=False)
        # add to data
        X.append(features)
        y.append(emotion)
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [0]:
X_train, X_test, y_train1, y_test1 = load_data(test_size=0.30)

# Tailor the input dimension to fit the model

In [9]:
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using extract_features() function
print("[+] Number of features:", X_train.shape[1])

lb = LabelEncoder()

y_train=np.array(y_train1)
y_test=np.array(y_test1)

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

print(x_traincnn.shape) 
print(x_testcnn.shape)
print(y_train.shape) 
print(y_test.shape)

[+] Number of training samples: 604
[+] Number of testing samples: 260
[+] Number of features: 40
(604, 40, 1)
(260, 40, 1)
(604, 5)
(260, 5)


# Train the model

Model 1 

In [56]:
model = Sequential()

model.add(Conv1D(32,3,padding='same',activation='relu',input_shape=(40,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))
model.add(Conv1D(32,3,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))
model.add(Conv1D(64,3,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))
model.add(Conv1D(64,3,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=5,activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

cnnhistory=model.fit(x_traincnn, np.array(y_train), batch_size=32, epochs=100, validation_data=(x_testcnn, np.array(y_test)))
print("Accuracy: {:.2f}%".format(cnnhistory.history['acc'][-1]*100))
print("Validation Accuracy: {:.2f}%".format(cnnhistory.history['val_acc'][-1]*100))


Train on 604 samples, validate on 260 samples
Epoch 1/100
604/604 [==============================] - 8s 13ms/step - loss: 4.7718 - acc: 0.2765 - val_loss: 1.8421 - val_acc: 0.2885
Epoch 2/100
604/604 [==============================] - 0s 344us/step - loss: 2.5216 - acc: 0.2616 - val_loss: 1.5266 - val_acc: 0.3423
Epoch 3/100
604/604 [==============================] - 0s 347us/step - loss: 2.1301 - acc: 0.2550 - val_loss: 1.6454 - val_acc: 0.2885
Epoch 4/100
604/604 [==============================] - 0s 346us/step - loss: 1.8018 - acc: 0.2831 - val_loss: 1.4807 - val_acc: 0.3385
Epoch 5/100
604/604 [==============================] - 0s 346us/step - loss: 1.7085 - acc: 0.2864 - val_loss: 1.4861 - val_acc: 0.3154
Epoch 6/100
604/604 [==============================] - 0s 361us/step - loss: 1.6031 - acc: 0.3212 - val_loss: 1.4821 - val_acc: 0.2808
Epoch 7/100
604/604 [==============================] - 0s 372us/step - loss: 1.5589 - acc: 0.3460 - val_loss: 1.4418 - val_acc: 0.3462
Epoch 8/10

Train new one

# Save the model

In [57]:
model_name = 'Emotion_Voice_Detection_Model1.h5'
model_path = os.path.join('/content', model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)


Saved trained model at /content/Emotion_Voice_Detection_Model1.h5 


Save the model to a json file

In [0]:
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)

## Prediction

Process the model for prediction

In [0]:

import pandas as pd
feature_test=extract_feature('filename_sad.wav', mfcc=True, chroma=False, mel=False)
livedf2 = feature_test
livedf2= pd.DataFrame(data=livedf2)
livedf2 = livedf2.stack().to_frame().T

In [76]:
livedf2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,-204.521686,67.135924,5.374407,15.67512,-6.406191,7.32221,-3.289051,3.839779,-1.570694,6.404727,1.85788,2.131837,0.491811,1.17804,-0.260022,3.017572,3.76518,2.88533,5.730032,7.221426,6.92007,9.646391,7.658868,7.259945,6.227987,5.43227,4.716178,2.922552,3.03751,2.165741,0.664761,1.398457,2.055024,2.014336,0.822774,2.907574,3.921482,2.361266,2.968615,3.139112


In [0]:
input_test= np.expand_dims(livedf2, axis=2)

In [78]:
input_test.shape

(1, 40, 1)

Load model and run prediction

In [79]:
from keras.models import model_from_json
json_file = open('model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/Emotion_Voice_Detection_Model1.h5")
print("Loaded model from disk")

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
#score = loaded_model.evaluate(, y_test, verbose=0)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))


preds = loaded_model.predict(input_test, 
                         batch_size=32, 
                         verbose=1)

Loaded model from disk
1/1 [==============================] - 6s 6s/step


In [0]:
preds1=preds.argmax(axis=1)

abc = preds1.astype(int).flatten()
predictions = (lb.inverse_transform((abc)))

preddf = pd.DataFrame({'predictedvalues': predictions})

In [81]:
preddf


,predictedvalues
0,sad
